# **Conceptos Fundamentales de azureml-core SDK v1**


## **1. `azureml-core`**
- `azureml-core` es el componente central del SDK de Azure Machine Learning. 
- Proporciona herramientas para gestionar áreas de trabajo, experimentos, pipelines y recursos computacionales. 
- Es esencial para interactuar con Azure ML desde un entorno de desarrollo local o remoto.

> https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core?view=azure-ml-py

### Instalación
Antes de comenzar, asegúrate de instalar el SDK:


In [3]:
# ! pip install azureml-core

## **2. Configuración Inicial**

### Crear un área de trabajo
El área de trabajo (`Workspace`) es el recurso central que organiza todos los recursos de Azure ML. Puedes crear una nueva área de trabajo o conectarte a una existente.

In [ ]:
from azureml.core import Workspace

# Crear un área de trabajo
ws = Workspace.create(name='mi-area-trabajo',
                      subscription_id='TU_SUBSCRIPTION_ID',  # Reemplaza con tu ID de suscripción
                      resource_group='mi-grupo-recursos',    # Reemplaza con tu grupo de recursos
                      create_resource_group=True,
                      location='eastus')                     # Reemplaza con tu región

# Guardar la configuración en un archivo config.json
ws.write_config()
print(f"Área de trabajo creada: {ws.name}")



### Conectarse a un área de trabajo existente
Si ya tienes un área de trabajo, puedes conectarte a ella utilizando el archivo `config.json` generado previamente.


In [ ]:
from azureml.core import Workspace

# Conectarse a un área de trabajo existente
ws = Workspace.from_config()
print(f"Conectado al área de trabajo: {ws.name}")


O indicando los datos de conexión:

In [ ]:
from azureml.core import Workspace

# Configuración manual del Workspace
subscription_id = "TU_SUBSCRIPTION_ID"
resource_group = "TU_GRUPO_DE_RECURSOS"
workspace_name = "TU_WORKSPACE"

# Conectarse al Workspace
ws = Workspace.get(
    name=workspace_name,
    subscription_id=subscription_id,
    resource_group=resource_group
)

# Verificar la conexión
print(f"Conectado al Workspace: {ws.name}")
print(f"Grupo de Recursos: {ws.resource_group}")
print(f"Suscripción: {ws.subscription_id}")

## **3. Experimentos**

### ¿Qué son los experimentos?
Un experimento (`Experiment`) es una colección de ejecuciones (runs) que permiten realizar pruebas y comparaciones entre diferentes modelos o configuraciones.

#### Ejemplo: Crear y ejecutar un experimento


In [ ]:
from azureml.core import Experiment

# Crear un experimento
experiment = Experiment(workspace=ws, name="mi-primer-experimento")

# Iniciar una ejecución
run = experiment.start_logging()

# Registrar métricas
run.log("precisión", 0.95)
run.log("pérdida", 0.05)

# Finalizar la ejecución
run.complete()
print(f"Ejecución completada: {run.id}")

## **4. Datos**

### Conjuntos de datos (`Dataset`)
Azure ML permite trabajar con conjuntos de datos (`Dataset`) para cargar y procesar datos de manera eficiente.

#### Ejemplo: Crear un conjunto de datos desde un archivo local


In [ ]:
from azureml.core import Dataset

# Subir un archivo local al área de trabajo
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./datos', target_path='datos')

# Crear un conjunto de datos
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'datos/mi-dataset.csv'))

# Mostrar las primeras filas del conjunto de datos
df = dataset.to_pandas_dataframe()
print(df.head())

## **5. Computación**

### Destinos de proceso (`ComputeTarget`)
Un destino de proceso (`ComputeTarget`) define dónde se ejecutan tus experimentos. Puede ser una máquina local, una máquina virtual o un clúster de GPU en la nube.

#### Ejemplo: Crear un clúster de proceso


In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Nombre del clúster
compute_name = "mi-cluster"

try:
    # Verificar si el clúster ya existe
    compute_target = AmlCompute(workspace=ws, name=compute_name)
    print("Clúster ya existe.")
except ComputeTargetException:
    # Crear un nuevo clúster
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2",  # Tipo de máquina virtual
        max_nodes=4                # Número máximo de nodos
    )
    compute_target = AmlCompute.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

print(f"Clúster creado: {compute_target.name}")

## **6. Entrenamiento de Modelos**

### Scripts de entrenamiento
Puedes usar scripts de Python para entrenar modelos y registrarlos en Azure ML.

#### Ejemplo: Entrenar un modelo simple
1. Crea un script de entrenamiento (`train.py`) con el siguiente contenido:


In [ ]:
# train.py
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import joblib

# Cargar datos
X, y = load_iris(return_X_y=True)

# Entrenar modelo
model = LogisticRegression()
model.fit(X, y)

# Guardar modelo
joblib.dump(model, "model.pkl")


2. Envía el script al clúster de proceso:


In [ ]:
from azureml.core import ScriptRunConfig

# Configurar el script de entrenamiento
src = ScriptRunConfig(
    source_directory="./scripts",
    script="train.py",
    compute_target=compute_target,
    environment=None  # Usa el entorno predeterminado
)

# Enviar el experimento
run = experiment.submit(src)
run.wait_for_completion(show_output=True)

print(f"Modelo entrenado: {run.id}")

## **7. Registro de Modelos**

### Registrar un modelo
Una vez entrenado, puedes registrar el modelo en Azure ML para su posterior uso.



In [ ]:
# Registrar el modelo
model = run.register_model(
    model_name="mi-modelo",
    model_path="model.pkl"
)

print(f"Modelo registrado: {model.name}, versión: {model.version}")

## **8. Despliegue de Modelos**

### Desplegar un modelo como servicio web
Puedes desplegar tu modelo como un servicio web en Azure Container Instances (ACI) o Azure Kubernetes Service (AKS).

#### Ejemplo: Desplegar en ACI


In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configurar el entorno de inferencia
inference_config = InferenceConfig(
    entry_script="score.py",  # Script de puntuación
    environment=None          # Usa el entorno predeterminado
)

# Configurar el servicio web
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Desplegar el modelo
service = Model.deploy(
    workspace=ws,
    name="mi-servicio",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)
print(f"Servicio desplegado: {service.scoring_uri}")

## **9. Referencias**

Para obtener más información sobre `azureml-core` y sus funcionalidades, consulta la documentación oficial:

- [Documentación oficial de azureml-core](https://learn.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py)
- [Guía de inicio rápido de Azure Machine Learning](https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources)
- [SDK de Azure Machine Learning en GitHub](https://github.com/Azure/azureml-sdk-for-python)
